<a href="https://colab.research.google.com/github/Fabiorsi1979/DS-Comunidade/blob/main/SQL_avan%C3%A7ado_Views_With.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Table VIEWS e WITH

#### As Tables Views só existem temporariamente e enquanto as querys estão sendo executadas. As funções WITH são usadas para auxiliar na composição de querys muito complexas, particionando e organizando os códigos.
~~~ sql
WITH parcelas AS (
    Select
          op.order_id,
          SUM ( op.payment_installments ) AS parcelas
    FROM order_payments op
    GROUP BY op.order_id
), produto AS (
  Select
        oi.order_id,
        COUNT ( oi.product_id ) AS produto
  FROM order_items oi
  GROUP BY oi.order_id
)
Select *
FROM parcelas p INNER JOIN produto po ON ( po.order_id = p.order_id )
LIMIT 10;
~~~

In [1]:
from google.colab import drive
drive.mount ('/content/drive/')

Mounted at /content/drive/


In [2]:
import sqlite3
from sqlite3 import Error
import pandas as pd

In [3]:
path = '/content/drive/MyDrive/db_olist.sqlite'

In [7]:
def consulta_with  (*args):
   try:
      conect = sqlite3.connect(path)
      cursor = conect.cursor()

      cursor.execute(query)
      consulta = cursor.fetchall()

      cursor.close()
      conect.close()

      dados = pd.DataFrame(data = consulta)

   except Error as e:
      print (f'Erro {e} conexão')
   return dados

##### 1. Crie uma consulta SQL usando a cláusula WITH para calcular o total de vendas para cada categoria e exiba o resultado.


In [5]:
query = """
WITH totalVendas As (
    SELECT
          op.order_id AS clientPag,
          op.payment_value AS Vendas
    FROM order_payments op
)
SELECT
    p.product_category_name,
    ROUND (SUM ( Vendas ),2 )AS vendasCategory
FROM order_items oi INNER JOIN products p    ON ( oi.product_id = p.product_id )
                    INNER JOIN totalVendas tV ON ( tV.clientPag = oi.order_id )
GROUP BY p.product_category_name
HAVING p.product_category_name IS NOT NULL
"""
resultado = consulta_with (query)
resultado.columns = ["Categoria","Total de Vendas"]
resultado


,Categoria,Total de Vendas
0,agro_industria_e_comercio,118730.61
1,alimentos,46428.67
2,alimentos_bebidas,27289.73
3,artes,30992.93
4,artes_e_artesanato,2326.17
...,...,...
68,sinalizacao_e_seguranca,68488.05
69,tablets_impressao_imagem,10042.93
70,telefonia,486882.05
71,telefonia_fixa,207010.26


##### 2. Crie uma consulta usando a cláusula WITH para calcular a receita total por mês e exiba o mês com a maior receita.

In [8]:
query = """
WITH receitaMes As (
     SELECT
          strftime ('%m',o.order_purchase_timestamp) As mes,
          ROUND (SUM ( op.payment_value ),2) AS totalVendas
     FROM order_payments op INNER JOIN orders o ON ( op.order_id = o.order_id )
     GROUP BY mes
)
SELECT
    mes,
    MAX (totalVendas) maiorVenda
FROM receitaMes rM
"""
resultado = consulta_with (query)
resultado.columns = ["Mês","Total de Vendas"]
resultado

,Mês,Total de Vendas
0,05,1746900.97


##### 3. Crie uma consulta usando duas cláusulas WITH para calcular o total de vendas por estado e, em seguida, por cidade, e exiba os resultados.

In [9]:
query = """
WITH tabVendas AS (
    SELECT
        op.order_id AS cod_tabVendas,
        op.payment_value AS pago_tabVendas
    FROM order_payments op
), tabCliente AS (
    SELECT
        c.customer_id AS cod_tabCliente,
        c.customer_city AS city_tabCliente,
        c.customer_state AS state_tabCliente
    FROM customer c
)
SELECT
    state_tabCliente AS UF,
    city_tabCliente AS Cid,
    ROUND( SUM ( pago_tabVendas) ,2) AS totalPago
FROM orders o INNER JOIN tabVendas tV ON ( tV.cod_tabVendas = o.order_id )
             INNER JOIN tabCliente tC ON ( tC.cod_tabCliente = o.customer_id )
GROUP BY UF, Cid
"""
resultado = consulta_with (query)
resultado.columns = ["UF", "Cidade","Valor Vendas"]
resultado

,UF,Cidade,Valor Vendas
0,AC,brasileia,139.53
1,AC,cruzeiro do sul,1127.06
2,AC,epitaciolandia,139.52
3,AC,manoel urbano,248.71
4,AC,porto acre,104.35
...,...,...,...
4305,TO,silvanopolis,273.88
4306,TO,sitio novo do tocantins,476.27
4307,TO,taguatinga,454.45
4308,TO,tocantinopolis,947.28


##### 4.Crie uma consulta SQL com as seguintes colunas:
    1) Categoria,
    2) Preço,
    3) Date limite de envio,
    4) Primeira compra,
    5) Número de dias
##### que o produto foi comprado a partir da primeira compra dentro da mesma categoria

In [13]:
query = """
WITH tabBase AS (
  SELECT
      p.product_category_name AS categoria,
      oi.price AS preco,
      oi.shipping_limit_date AS dtEnvio,
      FIRST_VALUE ( oi.shipping_limit_date ) OVER ( PARTITION BY p.product_category_name
                                                    ORDER BY oi.shipping_limit_date ASC ) AS pVenda
  FROM order_items oi INNER JOIN products p  ON ( p.product_id = oi.product_id )
  WHERE categoria IS NOT NULL
)
SELECT
    tB.categoria,
    tB.preco,
    tB.dtEnvio,
    tB.pVenda,
    CAST (JULIANDAY (dtEnvio) - JULIANDAY (pVenda) AS INTEGER) AS dias
FROM tabBase tB
"""
resultado = consulta_with (query)
resultado.columns = ["Categoria","Preço","Data limite envio","Primeira compra","Dias"]
resultado

,Categoria,Preço,Data limite envio,Primeira compra,Dias
0,agro_industria_e_comercio,21.99,2017-01-27 07:03:04,2017-01-27 07:03:04,0
1,agro_industria_e_comercio,21.99,2017-01-27 07:03:04,2017-01-27 07:03:04,0
2,agro_industria_e_comercio,21.99,2017-02-04 17:33:09,2017-01-27 07:03:04,8
3,agro_industria_e_comercio,21.99,2017-02-09 19:35:10,2017-01-27 07:03:04,13
4,agro_industria_e_comercio,21.99,2017-02-12 02:32:36,2017-01-27 07:03:04,15
...,...,...,...,...,...
111042,utilidades_domesticas,16.90,2018-09-12 13:24:27,2016-10-10 15:56:28,701
111043,utilidades_domesticas,75.99,2020-02-03 20:23:22,2016-10-10 15:56:28,1211
111044,utilidades_domesticas,69.99,2020-02-05 03:30:51,2016-10-10 15:56:28,1212
111045,utilidades_domesticas,99.99,2020-04-09 22:35:08,2016-10-10 15:56:28,1277
